# Process Analysis and Optimization

## Purpose

This notebook teaches you how to perform process analysis and parameter optimization. You'll learn parameter analysis, quality prediction, sensor analysis, and both single-objective and multi-objective optimization with interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Analyze process parameters (distribution, correlation, interactions)
- ✅ Predict quality from process parameters
- ✅ Analyze sensor data (ISPM, CT)
- ✅ Optimize process parameters (single-objective)
- ✅ Handle multi-objective optimization with Pareto fronts
- ✅ Visualize optimization progress and results

## Estimated Duration

60-90 minutes

---

## Overview

Process analysis and optimization are essential for improving manufacturing quality. The AM-QADF framework provides comprehensive capabilities:

- 📊 **Parameter Analysis**: Distribution, correlation, interaction analysis
- 🎯 **Quality Prediction**: Linear, polynomial, ML, neural network models
- 📡 **Sensor Analysis**: ISPM, CT sensor data analysis with trend and anomaly detection
- ⚡ **Single-Objective Optimization**: Genetic Algorithm, Particle Swarm, Gradient Descent, Bayesian
- 🎯 **Multi-Objective Optimization**: Pareto front analysis and trade-off visualization

Use the interactive widgets below to analyze and optimize processes - no coding required!


In [5]:
# Setup: Import required libraries
import sys
import time  
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple,
    HTML as WidgetHTML
)
from IPython.display import display, Markdown, HTML, Image, clear_output
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize, differential_evolution
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import process analysis classes
PROCESS_AVAILABLE = False
try:
    from am_qadf.analytics.process_analysis.parameter_analysis import ParameterAnalyzer
    from am_qadf.analytics.process_analysis.quality_analysis import QualityAnalyzer
    from am_qadf.analytics.process_analysis.optimization import ProcessOptimizer
    PROCESS_AVAILABLE = True
    print("✅ Process analysis classes available")
except ImportError as e:
    print(f"⚠️ Process analysis classes not available: {e} - using demo mode")

# MongoDB connection setup
INFRASTRUCTURE_AVAILABLE = False
mongo_client = None
voxel_storage = None
stl_client = None

try:
    from src.infrastructure.config import MongoDBConfig
    from src.infrastructure.database import MongoDBClient
    from am_qadf.voxel_domain import VoxelGridStorage
    from am_qadf.query import STLModelClient
    
    # Initialize MongoDB connection
    config = MongoDBConfig.from_env()
    if not config.username:
        config.username = os.getenv('MONGO_ROOT_USERNAME', 'admin')
    if not config.password:
        config.password = os.getenv('MONGO_ROOT_PASSWORD', 'password')
    
    mongo_client = MongoDBClient(config=config)
    if mongo_client.is_connected():
        voxel_storage = VoxelGridStorage(mongo_client=mongo_client)
        stl_client = STLModelClient(mongo_client=mongo_client)
        INFRASTRUCTURE_AVAILABLE = True
        print(f"✅ Connected to MongoDB: {config.database}")
    else:
        print("⚠️ MongoDB connection failed")
except Exception as e:
    print(f"⚠️ MongoDB not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
✅ Process analysis classes available
✅ Connected to MongoDB: am_qadf_data
✅ Setup complete!


## Interactive Process Analysis and Optimization Interface

Use the widgets below to analyze processes and optimize parameters. Select analysis mode, configure parameters, and visualize results interactively!


In [6]:
# Create Interactive Process Analysis and Optimization Interface

# Global state
analysis_results = {}
optimization_results = {}
process_data = {}
current_model_id = None
current_grid_id = None
loaded_grid_data = None
signal_arrays = {}

# ============================================
# Helper Functions for Demo Data
# ============================================

def generate_sample_process_data():
    """Generate sample process data for analysis."""
    np.random.seed(42)
    
    n_samples = 200
    
    # Process parameters
    laser_power = np.random.uniform(150, 300, n_samples)
    scan_speed = np.random.uniform(0.5, 2.0, n_samples)
    layer_thickness = np.random.uniform(0.02, 0.1, n_samples)
    hatch_spacing = np.random.uniform(0.05, 0.2, n_samples)
    
    # Quality metrics (correlated with parameters)
    quality = (
        0.4 * (laser_power - 150) / 150 +
        0.3 * (scan_speed - 0.5) / 1.5 +
        0.2 * (layer_thickness - 0.02) / 0.08 +
        0.1 * (hatch_spacing - 0.05) / 0.15 +
        np.random.normal(0, 0.05, n_samples)
    )
    quality = np.clip(quality, 0.0, 1.0)
    
    # Sensor data (time series)
    time_points = np.linspace(0, 1000, n_samples)
    ispm_temp = 200 + 50 * np.sin(2 * np.pi * time_points / 200) + np.random.normal(0, 5, n_samples)
    ct_density = 7.8 + 0.5 * np.sin(2 * np.pi * time_points / 150) + np.random.normal(0, 0.1, n_samples)
    
    return {
        'laser_power': laser_power,
        'scan_speed': scan_speed,
        'layer_thickness': layer_thickness,
        'hatch_spacing': hatch_spacing,
        'quality': quality,
        'time': time_points,
        'ispm_temp': ispm_temp,
        'ct_density': ct_density
    }

def demo_objective_function(params):
    """Demo objective function for optimization."""
    # Maximize quality (minimize negative quality)
    quality = (
        0.4 * (params[0] - 150) / 150 +
        0.3 * (params[1] - 0.5) / 1.5 +
        0.2 * (params[2] - 0.02) / 0.08 +
        0.1 * (params[3] - 0.05) / 0.15
    )
    return -quality  # Negative for minimization

# ============================================
# Top Panel: Data Source and Grid Selection
# ============================================

# Data source mode
data_source_label = widgets.HTML("<b>Data Source:</b>")
data_source_mode = RadioButtons(
    options=[('MongoDB', 'mongodb'), ('Sample Data', 'sample')],
    value='mongodb',
    description='Source:',
    style={'description_width': 'initial'}
)

# Model selection (for MongoDB)
model_label = widgets.HTML("<b>Model:</b>")
model_options = [("━━━ Select Model ━━━", None)]
if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")

model_dropdown = Dropdown(
    options=model_options,
    value=None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Grid type filter
grid_type_label = widgets.HTML("<b>Grid Type:</b>")
grid_type_filter = Dropdown(
    options=[
        ('All Grids', 'all'),
        ('Fused', 'fused'),
        ('Corrected', 'corrected'),
        ('Processed', 'processed'),
        ('Signal-Mapped', 'signal_mapped'),
        ('Raw', 'raw')
    ],
    value='fused',  # Default to fused grids
    description='Type:',
    style={'description_width': 'initial'}
)

# Grid selection (for MongoDB)
grid_label = widgets.HTML("<b>Grid:</b>")
grid_dropdown = Dropdown(
    options=[("━━━ Select Grid ━━━", None)],
    value=None,
    description='Grid:',
    style={'description_width': 'initial'},
    layout=Layout(width='500px')
)

load_grid_button = Button(
    description='Load Grid',
    button_style='info',
    icon='folder-open',
    layout=Layout(width='120px')
)

# Analysis mode
analysis_mode = RadioButtons(
    options=[
        ('Parameter Analysis', 'parameter'),
        ('Quality Prediction', 'quality'),
        ('Sensor Analysis', 'sensor'),
        ('Optimization', 'optimization')
    ],
    value='parameter',
    description='Mode:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute Analysis',
    button_style='success',
    icon='play',
    layout=Layout(width='160px')
)

optimize_button = Button(
    description='Optimize',
    button_style='primary',
    icon='cog',
    layout=Layout(width='120px')
)

top_panel = VBox([
    HBox([data_source_label, data_source_mode, analysis_mode]),
    HBox([model_label, model_dropdown, grid_type_label, grid_type_filter]),
    HBox([grid_label, grid_dropdown, load_grid_button]),
    HBox([execute_button, optimize_button])
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Analysis Configuration
# ============================================

# Parameter Analysis Section
param_label = widgets.HTML("<b>Parameter Analysis:</b>")
param_selector = SelectMultiple(
    options=[('Laser Power', 'laser_power'), ('Scan Speed', 'scan_speed'), 
             ('Layer Thickness', 'layer_thickness'), ('Hatch Spacing', 'hatch_spacing')],
    value=('laser_power', 'scan_speed'),
    description='Parameters:',
    style={'description_width': 'initial'}
)
param_analysis_type = RadioButtons(
    options=[('Distribution', 'distribution'), ('Correlation', 'correlation'), ('Interaction', 'interaction')],
    value='distribution',
    description='Type:',
    style={'description_width': 'initial'}
)
interaction_order = IntSlider(value=2, min=1, max=3, step=1, description='Interaction Order:', style={'description_width': 'initial'})

param_section = VBox([
    param_label,
    param_selector,
    param_analysis_type,
    interaction_order
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Quality Prediction Section
quality_label = widgets.HTML("<b>Quality Prediction:</b>")
prediction_method = Dropdown(
    options=[('Linear', 'linear'), ('Polynomial', 'polynomial'), ('ML', 'ml'), ('Neural Network', 'neural')],
    value='linear',
    description='Method:',
    style={'description_width': 'initial'}
)
feature_selector = SelectMultiple(
    options=[('Laser Power', 'laser_power'), ('Scan Speed', 'scan_speed'), 
             ('Layer Thickness', 'layer_thickness'), ('Hatch Spacing', 'hatch_spacing')],
    value=('laser_power', 'scan_speed', 'layer_thickness'),
    description='Features:',
    style={'description_width': 'initial'}
)
target_quality = Dropdown(
    options=[('Overall Quality', 'quality'), ('Density', 'density'), ('Surface Roughness', 'roughness')],
    value='quality',
    description='Target:',
    style={'description_width': 'initial'}
)
train_test_split = FloatSlider(value=0.7, min=0.1, max=0.9, step=0.1, description='Train/Test:', style={'description_width': 'initial'})

quality_section = VBox([
    quality_label,
    prediction_method,
    feature_selector,
    target_quality,
    train_test_split
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Sensor Analysis Section
sensor_label = widgets.HTML("<b>Sensor Analysis:</b>")
sensor_type = RadioButtons(
    options=[('ISPM', 'ispm'), ('CT', 'ct'), ('Both', 'both')],
    value='ispm',
    description='Sensor:',
    style={'description_width': 'initial'}
)
sensor_analysis_type = SelectMultiple(
    options=[('Trend', 'trend'), ('Anomaly', 'anomaly'), ('Correlation', 'correlation')],
    value=('trend',),
    description='Analysis:',
    style={'description_width': 'initial'}
)
time_window = IntSlider(value=100, min=1, max=1000, step=10, description='Time Window (s):', style={'description_width': 'initial'})

sensor_section = VBox([
    sensor_label,
    sensor_type,
    sensor_analysis_type,
    time_window
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Optimization Section
opt_label = widgets.HTML("<b>Optimization:</b>")
opt_type = RadioButtons(
    options=[('Single-Objective', 'single'), ('Multi-Objective', 'multi')],
    value='single',
    description='Type:',
    style={'description_width': 'initial'}
)
objective_function = Dropdown(
    options=[('Maximize Quality', 'quality'), ('Minimize Cost', 'cost'), ('Maximize Speed', 'speed')],
    value='quality',
    description='Objective:',
    style={'description_width': 'initial'}
)
opt_method = Dropdown(
    options=[('Genetic Algorithm', 'ga'), ('Particle Swarm', 'pso'), ('Gradient Descent', 'gradient'), ('Bayesian', 'bayesian')],
    value='ga',
    description='Method:',
    style={'description_width': 'initial'}
)
population_size = IntSlider(value=50, min=10, max=500, step=10, description='Population Size:', style={'description_width': 'initial'})
max_iterations = IntSlider(value=100, min=10, max=10000, step=10, description='Max Iterations:', style={'description_width': 'initial'})
convergence_tolerance = FloatSlider(value=0.001, min=0.0001, max=0.1, step=0.0001, description='Tolerance:', style={'description_width': 'initial'})

# Parameter bounds (simplified)
param_bounds_label = widgets.HTML("<b>Parameter Bounds:</b>")
laser_power_min = FloatSlider(value=150, min=100, max=200, step=10, description='Laser Power Min:', style={'description_width': 'initial'})
laser_power_max = FloatSlider(value=300, min=250, max=400, step=10, description='Laser Power Max:', style={'description_width': 'initial'})

opt_section = VBox([
    opt_label,
    opt_type,
    objective_function,
    opt_method,
    population_size,
    max_iterations,
    convergence_tolerance,
    param_bounds_label,
    laser_power_min,
    laser_power_max
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Show/hide sections based on mode
def update_mode_sections(change):
    """Show/hide sections based on analysis mode."""
    mode = change['new']
    param_section.layout.display = 'none'
    quality_section.layout.display = 'none'
    sensor_section.layout.display = 'none'
    opt_section.layout.display = 'none'
    
    if mode == 'parameter':
        param_section.layout.display = 'flex'
    elif mode == 'quality':
        quality_section.layout.display = 'flex'
    elif mode == 'sensor':
        sensor_section.layout.display = 'flex'
    elif mode == 'optimization':
        opt_section.layout.display = 'flex'

analysis_mode.observe(update_mode_sections, names='value')
update_mode_sections({'new': analysis_mode.value})

# ============================================
# Helper Functions for MongoDB
# ============================================

def update_grid_dropdown(change=None):
    """Update grid dropdown when model or grid type changes."""
    global current_model_id
    
    model_id = model_dropdown.value
    grid_type = grid_type_filter.value
    
    if not model_id:
        grid_dropdown.options = [("━━━ Select Grid ━━━", None)]
        return
    
    current_model_id = model_id
    
    if not voxel_storage:
        grid_dropdown.options = [("━━━ MongoDB not available ━━━", None)]
        return
    
    try:
        # Get all grids for this model
        grids = voxel_storage.list_grids(model_id=model_id, limit=100)
        
        grid_options = [("━━━ Select Grid ━━━", None)]
        for grid in grids:
            metadata = grid.get('metadata', {})
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            # Determine grid type
            is_fused = config_meta.get('fusion_applied', False)
            is_corrected = config_meta.get('correction_applied', False)
            is_processed = config_meta.get('processing_applied', False)
            has_signals = len(grid.get('available_signals', [])) > 0
            
            grid_type_match = False
            if grid_type == 'all':
                grid_type_match = True
            elif grid_type == 'fused' and is_fused:
                grid_type_match = True
            elif grid_type == 'corrected' and is_corrected:
                grid_type_match = True
            elif grid_type == 'processed' and is_processed:
                grid_type_match = True
            elif grid_type == 'signal_mapped' and has_signals and not is_corrected and not is_processed and not is_fused:
                grid_type_match = True
            elif grid_type == 'raw' and not has_signals:
                grid_type_match = True
            
            if grid_type_match:
                grid_id = grid.get('grid_id', str(grid.get('_id', '')))
                grid_name = grid.get('grid_name', 'Unknown')
                n_signals = len(grid.get('available_signals', []))
                
                # Build status label
                status_parts = []
                if is_fused:
                    status_parts.append('fused')
                if is_corrected:
                    status_parts.append('corrected')
                if is_processed:
                    status_parts.append('processed')
                if has_signals and not status_parts:
                    status_parts.append('mapped')
                if not status_parts:
                    status_parts.append('raw')
                
                status_str = ', '.join(status_parts)
                label = f"{grid_name} ({n_signals} signal(s), {status_str}) ({grid_id[:8]}...)"
                grid_options.append((label, grid_id))
        
        if len(grid_options) == 1:
            grid_options.append(("No grids found matching filter", None))
        
        grid_dropdown.options = grid_options
    except Exception as e:
        grid_dropdown.options = [("━━━ Error loading grids ━━━", None)]
        print(f"⚠️ Error loading grids: {e}")

def load_grid_from_mongodb(button):
    """Load selected grid from MongoDB."""
    global current_model_id, current_grid_id, loaded_grid_data, signal_arrays, operation_start_time
    
    if not voxel_storage or not grid_dropdown.value:
        log_message("Please select a grid to load", 'warning')
        update_status("Please select a grid", 0)
        return
    
    grid_id = grid_dropdown.value
    current_grid_id = grid_id
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with process_logs:
        clear_output(wait=True)
    
    log_message(f"Loading grid from MongoDB: {grid_id[:8]}...", 'info')
    update_status("Loading grid from MongoDB...", 0)
    
    try:
        # Load grid from MongoDB
        log_message("Connecting to MongoDB...", 'info')
        grid_data = voxel_storage.load_voxel_grid(grid_id=grid_id)
        
        if not grid_data:
            log_message("Failed to load grid from MongoDB", 'error')
            update_status("Failed to load grid", 0)
            return
        
        # Extract data from dictionary
        signal_arrays = grid_data.get('signal_arrays', {})
        metadata = grid_data.get('metadata', {})
        grid_name = grid_data.get('grid_name', 'Unknown')
        
        # Store loaded data
        loaded_grid_data = {
            'grid_data': grid_data,
            'metadata': metadata,
            'signal_arrays': signal_arrays
        }
        
        # Update parameter selectors with available signals
        if signal_arrays:
            # Get signal names, excluding metadata signals like 'fused'
            signal_names = [name for name in signal_arrays.keys() 
                          if name not in ['fused', 'combined', 'merged']]
            
            # If no signals after filtering, use all signals
            if not signal_names:
                signal_names = list(signal_arrays.keys())
            
            # Also check metadata for available_signals list
            if metadata and 'available_signals' in metadata:
                available_signal_list = metadata.get('available_signals', [])
                # Use signals that exist in both signal_arrays and available_signals
                signal_names = [name for name in available_signal_list if name in signal_arrays]
                # If still empty, fall back to signal_arrays keys
                if not signal_names:
                    signal_names = [name for name in signal_arrays.keys() 
                                  if name not in ['fused', 'combined', 'merged']]
                    if not signal_names:
                        signal_names = list(signal_arrays.keys())
            
            # Create options for parameter selector with better display names
            def format_signal_name(name):
                """Format signal name for display."""
                # Replace underscores with spaces and title case
                formatted = name.replace('_', ' ').title()
                # Handle common abbreviations
                formatted = formatted.replace('Ispm', 'ISPM')
                formatted = formatted.replace('Ct', 'CT')
                return formatted
            
            param_options = [(format_signal_name(name), name) for name in signal_names]
            param_selector.options = param_options
            # Set default selection to first few signals
            if len(signal_names) > 0:
                param_selector.value = tuple(signal_names[:min(2, len(signal_names))])
            
            # Update feature selector for quality prediction
            feature_selector.options = param_options
            if len(signal_names) > 0:
                feature_selector.value = tuple(signal_names[:min(3, len(signal_names))])
        
        log_message(f"Grid loaded: {grid_name} ({len(signal_arrays)} signal(s))", 'success')
        
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            log_message(f"Grid loading completed in {elapsed:.2f}s", 'success')
        
        update_status(f"Grid loaded: {grid_name}", 100)
        
    except Exception as e:
        log_message(f"Error loading grid: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        update_status("Error loading grid", 0)

# Function to update UI based on data source mode
def update_data_source_mode(change):
    """Show/hide MongoDB widgets based on data source mode."""
    if change['new'] == 'mongodb':
        model_dropdown.layout.display = 'flex'
        grid_type_filter.layout.display = 'flex'
        grid_dropdown.layout.display = 'flex'
        load_grid_button.layout.display = 'flex'
    else:
        model_dropdown.layout.display = 'none'
        grid_type_filter.layout.display = 'none'
        grid_dropdown.layout.display = 'none'
        load_grid_button.layout.display = 'none'

# Connect events
data_source_mode.observe(update_data_source_mode, names='value')
update_data_source_mode({'new': data_source_mode.value})
model_dropdown.observe(update_grid_dropdown, names='value')
grid_type_filter.observe(update_grid_dropdown, names='value')
load_grid_button.on_click(load_grid_from_mongodb)

left_panel = VBox([
    param_section,
    quality_section,
    sensor_section,
    opt_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('Results', 'results'), ('Optimization', 'optimization'), ('Pareto', 'pareto'), ('Comparison', 'comparison')],
    value='results',
    description='View:',
    style={'description_width': 'initial'}
)

viz_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Analysis Visualization</h3>"),
    viz_mode,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Results
# ============================================

# Parameter Statistics
param_stats_label = widgets.HTML("<b>Parameter Statistics:</b>")
param_stats_display = widgets.HTML("No analysis performed yet")
param_stats_section = VBox([
    param_stats_label,
    param_stats_display
], layout=Layout(padding='5px'))

# Quality Metrics
quality_metrics_label = widgets.HTML("<b>Quality Metrics:</b>")
quality_metrics_display = widgets.HTML("No quality metrics")
quality_metrics_section = VBox([
    quality_metrics_label,
    quality_metrics_display
], layout=Layout(padding='5px'))

# Optimization Results
opt_results_label = widgets.HTML("<b>Optimization Results:</b>")
opt_results_display = widgets.HTML("No optimization results")
opt_results_section = VBox([
    opt_results_label,
    opt_results_display
], layout=Layout(padding='5px'))

# Pareto Solutions
pareto_label = widgets.HTML("<b>Pareto Solutions:</b>")
pareto_display = widgets.HTML("No Pareto solutions")
pareto_section = VBox([
    pareto_label,
    pareto_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
export_results_button = Button(description='Export Results', button_style='', layout=Layout(width='150px'))
export_config_button = Button(description='Export Config', button_style='', layout=Layout(width='150px'))
export_pareto_button = Button(description='Export Pareto', button_style='', layout=Layout(width='150px'))
save_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))

export_section = VBox([
    export_label,
    export_results_button,
    export_config_button,
    export_pareto_button,
    save_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    param_stats_section,
    quality_metrics_section,
    opt_results_section,
    pareto_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress with Logging
# ============================================

# Status display widget
current_operation = WidgetHTML(value='<b>Status:</b> Ready to analyze')

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Process analysis logs output
process_logs = Output(layout=Layout(height='200px', border='1px solid #ccc', overflow_y='auto'))

# Initialize logs
with process_logs:
    display(HTML("<p><i>Process analysis logs will appear here...</i></p>"))

# Bottom status bar (shows Status | Progress | Time)
bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Info display (kept for backward compatibility)
info_display = widgets.HTML("")

# Enhanced bottom panel
bottom_panel = VBox([
    current_operation,
    progress_bar,
    WidgetHTML("<b>Process Analysis Logs:</b>"),
    process_logs,
    WidgetHTML("<hr>"),
    bottom_status,
    bottom_progress,
    info_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# Keep old status_display for backward compatibility (will be updated by logging functions)
status_display = current_operation

# Global time tracking
operation_start_time = None

# ============================================
# Logging Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the process analysis logs with timestamp and emoji."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    with process_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display and progress."""
    global operation_start_time
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {time.strftime("%M:%S", time.gmtime(elapsed))}'
        else:
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> 0:00'

# ============================================
# Analysis Functions
# ============================================

def execute_analysis(button):
    """Execute process analysis based on current settings."""
    global analysis_results, process_data, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs and start
    with process_logs:
        clear_output(wait=True)
    
    log_message("Starting process analysis...", 'info')
    update_status("Initializing analysis...", 0)
    info_display.value = ""
    
    try:
        mode = analysis_mode.value
        log_message(f"Analysis mode: {mode}", 'info')
        update_status("Preparing data...", 20)
        
        # Load data based on mode
        if data_source_mode.value == 'mongodb':
            if not loaded_grid_data or not signal_arrays:
                log_message("Please load a grid from MongoDB first", 'warning')
                update_status("Please load a grid", 0)
                return
            
            # Extract data from loaded grid
            metadata = loaded_grid_data.get('metadata', {})
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            # Convert signal arrays to process data format
            # Flatten 3D arrays to 1D for analysis
            process_data = {}
            for signal_name, signal_array in signal_arrays.items():
                process_data[signal_name] = signal_array.flatten()
            
            # Extract process parameters from metadata if available
            if config_meta.get('process_parameters'):
                process_params = config_meta.get('process_parameters', {})
                for param_name, param_value in process_params.items():
                    if isinstance(param_value, (int, float)):
                        process_data[param_name] = np.full(len(list(signal_arrays.values())[0].flatten()), param_value)
            
            # Extract quality metrics if available
            if config_meta.get('quality_metrics'):
                quality_metrics = config_meta.get('quality_metrics', {})
                for metric_name, metric_value in quality_metrics.items():
                    if isinstance(metric_value, (int, float)):
                        process_data[metric_name] = np.full(len(list(signal_arrays.values())[0].flatten()), metric_value)
            
            # Add time array if not present (for sensor analysis)
            if 'time' not in process_data and len(list(signal_arrays.values())) > 0:
                n_points = len(list(signal_arrays.values())[0].flatten())
                process_data['time'] = np.linspace(0, n_points, n_points)
            
            log_message(f"Loaded {len(signal_arrays)} signal(s) from grid", 'success')
            update_status("Data loaded", 30)
        else:
            # Use sample data
            log_message("Using sample process data", 'info')
            process_data = generate_sample_process_data()
            # Reset parameter selectors to default sample data options
            param_selector.options = [
                ('Laser Power', 'laser_power'), 
                ('Scan Speed', 'scan_speed'), 
                ('Layer Thickness', 'layer_thickness'), 
                ('Hatch Spacing', 'hatch_spacing')
            ]
            param_selector.value = ('laser_power', 'scan_speed')
            feature_selector.options = [
                ('Laser Power', 'laser_power'), 
                ('Scan Speed', 'scan_speed'), 
                ('Layer Thickness', 'layer_thickness'), 
                ('Hatch Spacing', 'hatch_spacing')
            ]
            feature_selector.value = ('laser_power', 'scan_speed', 'layer_thickness')
            log_message("Sample data generated", 'success')
            update_status("Sample data ready", 30)
        
        update_status("Running analysis...", 40)
        
        # Perform analysis based on mode
        if mode == 'parameter':
            log_message("Performing parameter analysis...", 'info')
            results = perform_parameter_analysis()
        elif mode == 'quality':
            log_message("Performing quality prediction...", 'info')
            results = perform_quality_prediction()
        elif mode == 'sensor':
            log_message("Performing sensor analysis...", 'info')
            results = perform_sensor_analysis()
        else:
            log_message("Unknown analysis mode", 'warning')
            results = {}
        
        # Check for errors in results
        if isinstance(results, dict) and results.get('type') == 'error':
            log_message(f"Analysis failed: {results.get('message', 'Unknown error')}", 'error')
            info_display.value = f"<span style='color: red;'>❌ Error: {results.get('message', 'Unknown error')}</span>"
            update_status("Analysis failed", 0)
            return
        
        analysis_results = results
        log_message(f"{mode.capitalize()} analysis completed", 'success')
        update_status("Processing results...", 80)
        
        # Update displays
        update_results_display()
        update_visualization()
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Analysis completed in {total_time:.2f}s", 'success')
        else:
            log_message("Analysis completed successfully", 'success')
        
        update_status("Analysis completed", 100)
        grid_name = loaded_grid_data.get('grid_data', {}).get('grid_name', 'Sample Data') if data_source_mode.value == 'mongodb' and loaded_grid_data else 'Sample Data'
        info_display.value = f"<p>Mode: <b>{mode}</b> | Data: <b>{grid_name}</b></p>"
        
    except Exception as e:
        log_message(f"Error during analysis: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        update_status("Error during analysis", 0)

def perform_parameter_analysis():
    """Perform parameter analysis."""
    selected_params = list(param_selector.value)
    
    # Filter to only parameters that exist in process_data
    available_params = [p for p in selected_params if p in process_data]
    
    if not available_params:
        # Show what parameters are actually available
        available_keys = [k for k in process_data.keys() if k not in ['time', 'x', 'y', 'z'] and not isinstance(process_data[k], str)]
        if available_keys:
            available_str = ', '.join(available_keys[:10])  # Show first 10
            if len(available_keys) > 10:
                available_str += f', ... ({len(available_keys)} total)'
            return {
                'type': 'error',
                'message': f'No selected parameters found in data. Available parameters/signals: {available_str}. Please select parameters that exist in the loaded grid.'
            }
        else:
            return {
                'type': 'error',
                'message': 'No parameters or signals found in the loaded grid data.'
            }
    
    if param_analysis_type.value == 'distribution':
        return {
            'type': 'distribution',
            'parameters': available_params,
            'distributions': {p: {'mean': np.mean(process_data[p]), 'std': np.std(process_data[p])} 
                            for p in available_params}
        }
    elif param_analysis_type.value == 'correlation':
        correlations = {}
        for i, p1 in enumerate(available_params):
            for p2 in available_params[i+1:]:
                if p1 in process_data and p2 in process_data:
                    try:
                        corr = np.corrcoef(process_data[p1], process_data[p2])[0, 1]
                        if not np.isnan(corr):
                            correlations[f'{p1}-{p2}'] = corr
                    except:
                        pass
        return {
            'type': 'correlation',
            'parameters': available_params,
            'correlations': correlations
        }
    else:  # interaction
        return {
            'type': 'interaction',
            'parameters': available_params,
            'order': interaction_order.value
        }

def perform_quality_prediction():
    """Perform quality prediction."""
    features = list(feature_selector.value)
    
    # Filter to only features that exist in process_data
    available_features = [f for f in features if f in process_data]
    
    if not available_features:
        return {
            'type': 'error',
            'message': 'No selected features found in data. Please select features that exist in the loaded grid.'
        }
    
    # Check if target exists, if not use first available signal as proxy
    if 'quality' not in process_data:
        # Use first available signal as quality proxy
        available_signals = [k for k in process_data.keys() if k not in ['time', 'x', 'y', 'z']]
        if available_signals:
            quality_key = available_signals[0]
        else:
            return {
                'type': 'error',
                'message': 'No quality metric or signals found in data.'
            }
    else:
        quality_key = 'quality'
    
    # Simple linear regression
    try:
        X = np.column_stack([process_data[f] for f in available_features])
        y = process_data[quality_key]
    except KeyError as e:
        return {
            'type': 'error',
            'message': f'Missing data for analysis: {str(e)}'
        }
    
    # Train/test split
    n_train = int(len(X) * train_test_split.value)
    X_train, X_test = X[:n_train], X[n_train:]
    y_train, y_test = y[:n_train], y[n_train:]
    
    # Simple linear model
    if prediction_method.value == 'linear':
        coeffs = np.linalg.lstsq(X_train, y_train, rcond=None)[0]
        y_pred = X_test @ coeffs
        r2 = 1 - np.sum((y_test - y_pred)**2) / np.sum((y_test - np.mean(y_test))**2)
        rmse = np.sqrt(np.mean((y_test - y_pred)**2))
        
        return {
            'method': 'linear',
            'features': features,
            'r2': r2,
            'rmse': rmse,
            'y_test': y_test,
            'y_pred': y_pred
        }
    return {}

def perform_sensor_analysis():
    """Perform sensor analysis."""
    sensor = sensor_type.value
    
    # Find available sensor signals
    available_sensors = {}
    for key in process_data.keys():
        if 'ispm' in key.lower() or 'temp' in key.lower():
            available_sensors['ispm'] = key
        elif 'ct' in key.lower() or 'density' in key.lower():
            available_sensors['ct'] = key
    
    # Select sensor data
    if sensor == 'ispm' or sensor == 'both':
        if 'ispm' in available_sensors:
            sensor_data = process_data[available_sensors['ispm']]
        elif 'ispm_temp' in process_data:
            sensor_data = process_data['ispm_temp']
        else:
            # Use first available signal as fallback
            available_signals = [k for k in process_data.keys() if k not in ['time', 'x', 'y', 'z']]
            if available_signals:
                sensor_data = process_data[available_signals[0]]
            else:
                return {
                    'type': 'error',
                    'message': 'No ISPM sensor data found in the loaded grid.'
                }
    else:
        if 'ct' in available_sensors:
            sensor_data = process_data[available_sensors['ct']]
        elif 'ct_density' in process_data:
            sensor_data = process_data['ct_density']
        else:
            # Use first available signal as fallback
            available_signals = [k for k in process_data.keys() if k not in ['time', 'x', 'y', 'z']]
            if available_signals:
                sensor_data = process_data[available_signals[0]]
            else:
                return {
                    'type': 'error',
                    'message': 'No CT sensor data found in the loaded grid.'
                }
    
    # Get time array
    if 'time' in process_data:
        time_data = process_data['time']
    else:
        time_data = np.arange(len(sensor_data))
    
    results = {
        'sensor': sensor,
        'data': sensor_data,
        'time': time_data
    }
    
    if 'trend' in sensor_analysis_type.value:
        # Simple trend detection
        try:
            coeffs = np.polyfit(time_data, sensor_data, 1)
            results['trend'] = {'slope': coeffs[0], 'intercept': coeffs[1]}
        except:
            results['trend'] = {'slope': 0, 'intercept': np.mean(sensor_data)}
    
    if 'anomaly' in sensor_analysis_type.value:
        # Simple anomaly detection (Z-score)
        mean = np.mean(sensor_data)
        std = np.std(sensor_data)
        if std > 0:
            z_scores = np.abs((sensor_data - mean) / std)
            anomalies = z_scores > 2
            results['anomalies'] = {'count': np.sum(anomalies), 'indices': np.where(anomalies)[0]}
        else:
            results['anomalies'] = {'count': 0, 'indices': np.array([])}
    
    return results

def run_optimization(button):
    """Run optimization."""
    global optimization_results, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with process_logs:
        clear_output(wait=True)
    
    log_message("Starting optimization...", 'info')
    update_status("Initializing optimization...", 0)
    info_display.value = ""
    
    try:
        opt_type_val = opt_type.value
        opt_method_val = opt_method.value if opt_type_val == 'single' else 'pareto'
        
        log_message(f"Optimization type: {opt_type_val}, Method: {opt_method_val}", 'info')
        
        # Define bounds
        bounds = [
            (laser_power_min.value, laser_power_max.value),
            (0.5, 2.0),  # scan_speed
            (0.02, 0.1),  # layer_thickness
            (0.05, 0.2)  # hatch_spacing
        ]
        
        log_message(f"Parameter bounds: Power [{bounds[0][0]}-{bounds[0][1]}], Speed [{bounds[1][0]}-{bounds[1][1]}]", 'info')
        update_status("Setting up optimization...", 20)
        
        if opt_type_val == 'single':
            # Single-objective optimization
            log_message(f"Running single-objective optimization (method: {opt_method_val})...", 'info')
            log_message(f"Max iterations: {max_iterations.value}, Population size: {population_size.value}", 'info')
            update_status("Running optimization...", 30)
            
            if opt_method.value == 'ga':
                log_message("Using Genetic Algorithm (Differential Evolution)...", 'info')
                result = differential_evolution(
                    demo_objective_function,
                    bounds,
                    maxiter=max_iterations.value,
                    popsize=population_size.value,
                    tol=convergence_tolerance.value,
                    seed=42
                )
            else:
                log_message("Using Gradient Descent (L-BFGS-B)...", 'info')
                # Simple gradient descent
                x0 = [(b[0] + b[1]) / 2 for b in bounds]
                result = minimize(
                    demo_objective_function,
                    x0,
                    method='L-BFGS-B',
                    bounds=bounds,
                    options={'maxiter': max_iterations.value}
                )
            
            iterations = result.nit if hasattr(result, 'nit') else max_iterations.value
            log_message(f"Optimization converged after {iterations} iterations", 'success')
            log_message(f"Optimal value: {-result.fun:.4f}", 'success')
            
            optimization_results = {
                'type': 'single',
                'optimal_params': result.x,
                'optimal_value': -result.fun,  # Negative because we minimize negative quality
                'success': result.success,
                'iterations': iterations
            }
        else:
            # Multi-objective (simplified - generate Pareto front)
            log_message("Running multi-objective optimization (Pareto front)...", 'info')
            update_status("Generating Pareto solutions...", 30)
            
            np.random.seed(42)
            n_solutions = 20
            pareto_solutions = []
            for i in range(n_solutions):
                if (i + 1) % 5 == 0:
                    log_message(f"Generating solution {i+1}/{n_solutions}...", 'info')
                    update_status(f"Generating Pareto solutions... {i+1}/{n_solutions}", 30 + int(40 * (i+1) / n_solutions))
                
                params = [np.random.uniform(b[0], b[1]) for b in bounds]
                obj1 = -demo_objective_function(params)  # Quality
                obj2 = np.sum(params) / 1000  # Cost (simplified)
                pareto_solutions.append({'params': params, 'obj1': obj1, 'obj2': obj2})
            
            log_message(f"Generated {len(pareto_solutions)} Pareto solutions", 'success')
            
            optimization_results = {
                'type': 'multi',
                'pareto_solutions': pareto_solutions
            }
        
        update_status("Processing results...", 80)
        
        # Update displays
        update_optimization_display()
        
        # Switch to optimization view if available
        if optimization_results['type'] == 'single':
            viz_mode.value = 'optimization'
        elif optimization_results['type'] == 'multi':
            viz_mode.value = 'pareto'
        
        update_visualization()
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Optimization completed in {total_time:.2f}s", 'success')
        else:
            log_message("Optimization completed successfully", 'success')
        
        update_status("Optimization completed", 100)
        info_display.value = f"<p>Type: <b>{opt_type.value}</b> | Method: <b>{opt_method.value}</b></p>"
        
    except Exception as e:
        log_message(f"Error during optimization: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        update_status("Error during optimization", 0)

def update_results_display():
    """Update results displays."""
    global analysis_results
    
    if not analysis_results:
        return
    
    mode = analysis_mode.value
    
    # Parameter statistics
    if mode == 'parameter' and 'distributions' in analysis_results:
        stats_html = "<table border='1' style='border-collapse: collapse; width: 100%;'><tr><th>Parameter</th><th>Mean</th><th>Std</th></tr>"
        for param, dist in analysis_results['distributions'].items():
            stats_html += f"<tr><td>{param}</td><td>{dist['mean']:.2f}</td><td>{dist['std']:.2f}</td></tr>"
        stats_html += "</table>"
        param_stats_display.value = stats_html
    
    # Quality metrics
    if mode == 'quality' and 'r2' in analysis_results:
        metrics_html = f"<p><b>R² Score:</b> {analysis_results['r2']:.4f}</p>"
        metrics_html += f"<p><b>RMSE:</b> {analysis_results['rmse']:.4f}</p>"
        metrics_html += f"<p><b>Method:</b> {analysis_results['method']}</p>"
        quality_metrics_display.value = metrics_html

def update_optimization_display():
    """Update optimization results display."""
    global optimization_results
    
    if not optimization_results:
        return
    
    if optimization_results['type'] == 'single':
        opt_html = f"<p><b>Optimal Value:</b> {optimization_results['optimal_value']:.4f}</p>"
        opt_html += f"<p><b>Parameters:</b></p><ul>"
        param_names = ['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing']
        for name, value in zip(param_names, optimization_results['optimal_params']):
            opt_html += f"<li>{name}: {value:.4f}</li>"
        opt_html += "</ul>"
        opt_html += f"<p><b>Success:</b> {optimization_results['success']}</p>"
        opt_html += f"<p><b>Iterations:</b> {optimization_results['iterations']}</p>"
        opt_results_display.value = opt_html
    else:
        pareto_html = f"<p><b>Pareto Solutions:</b> {len(optimization_results['pareto_solutions'])}</p>"
        pareto_display.value = pareto_html

def show_figure(fig):
    """Display matplotlib figure in Jupyter Output widget."""
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=100, bbox_inches='tight')
    buf.seek(0)
    display(Image(buf.getvalue()))
    plt.close(fig)
    
def update_visualization():
    """Update visualization display."""
    global analysis_results, optimization_results, process_data
    
    with viz_output:
        clear_output(wait=True)
        
        mode = analysis_mode.value
        viz = viz_mode.value
        
        # Handle optimization visualizations first (they work regardless of current mode)
        if viz == 'optimization' and optimization_results:
            if optimization_results['type'] == 'single':
                fig, ax = plt.subplots(figsize=(10, 6))
                # Simulated optimization history
                iterations = range(optimization_results['iterations'])
                history = [optimization_results['optimal_value'] * (1 - 0.5 * np.exp(-i/20)) for i in iterations]
                ax.plot(iterations, history, linewidth=2, label='Objective Value')
                ax.axhline(y=optimization_results['optimal_value'], color='r', linestyle='--', label='Optimal')
                ax.set_xlabel('Iteration')
                ax.set_ylabel('Objective Value')
                ax.set_title('Optimization Progress')
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                show_figure(fig)
            elif optimization_results['type'] == 'multi':
                if 'pareto_solutions' in optimization_results:
                    fig, ax = plt.subplots(figsize=(10, 6))
                    solutions = optimization_results['pareto_solutions']
                    obj1_vals = [s['obj1'] for s in solutions]
                    obj2_vals = [s['obj2'] for s in solutions]
                    ax.scatter(obj1_vals, obj2_vals, alpha=0.6, s=50)
                    ax.set_xlabel('Objective 1 (Quality)')
                    ax.set_ylabel('Objective 2 (Cost)')
                    ax.set_title('Pareto Front')
                    ax.grid(True, alpha=0.3)
                    plt.tight_layout()
                    show_figure(fig)
            return
        
        elif viz == 'pareto' and optimization_results and optimization_results['type'] == 'multi':
            if 'pareto_solutions' in optimization_results:
                fig, ax = plt.subplots(figsize=(10, 6))
                solutions = optimization_results['pareto_solutions']
                obj1_vals = [s['obj1'] for s in solutions]
                obj2_vals = [s['obj2'] for s in solutions]
                ax.scatter(obj1_vals, obj2_vals, alpha=0.6, s=50)
                ax.set_xlabel('Objective 1 (Quality)')
                ax.set_ylabel('Objective 2 (Cost)')
                ax.set_title('Pareto Front')
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                show_figure(fig)
            return
        
        # Handle analysis visualizations
        if mode == 'parameter' and viz == 'results':
            if 'distributions' in analysis_results:
                fig, axes = plt.subplots(1, len(analysis_results['parameters']), figsize=(5 * len(analysis_results['parameters']), 4))
                if len(analysis_results['parameters']) == 1:
                    axes = [axes]
                
                for idx, param in enumerate(analysis_results['parameters']):
                    if param in process_data:
                        axes[idx].hist(process_data[param], bins=30, alpha=0.7, edgecolor='black')
                        axes[idx].set_title(f'{param} Distribution')
                        axes[idx].set_xlabel('Value')
                        axes[idx].set_ylabel('Frequency')
                
                plt.tight_layout()
                show_figure(fig)
            elif 'correlations' in analysis_results:
                fig, ax = plt.subplots(figsize=(8, 6))
                params = analysis_results['parameters']
                corr_matrix = np.eye(len(params))
                for i, p1 in enumerate(params):
                    for j, p2 in enumerate(params):
                        if i != j:
                            key = f'{p1}-{p2}' if f'{p1}-{p2}' in analysis_results['correlations'] else f'{p2}-{p1}'
                            if key in analysis_results['correlations']:
                                corr_matrix[i, j] = analysis_results['correlations'][key]
                
                im = ax.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
                ax.set_xticks(range(len(params)))
                ax.set_yticks(range(len(params)))
                ax.set_xticklabels(params, rotation=45, ha='right')
                ax.set_yticklabels(params)
                ax.set_title('Parameter Correlation Matrix')
                plt.colorbar(im, ax=ax, label='Correlation')
                plt.tight_layout()
                show_figure(fig)
        
        elif mode == 'quality' and viz == 'results':
            if 'y_test' in analysis_results:
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.scatter(analysis_results['y_test'], analysis_results['y_pred'], alpha=0.6)
                ax.plot([0, 1], [0, 1], 'r--', label='Perfect Prediction')
                ax.set_xlabel('Actual Quality')
                ax.set_ylabel('Predicted Quality')
                ax.set_title(f'Quality Prediction (R²={analysis_results.get("r2", 0):.3f})')
                ax.legend()
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                show_figure(fig)
        
        elif mode == 'sensor' and viz == 'results':
            if 'data' in analysis_results:
                fig, ax = plt.subplots(figsize=(12, 5))
                time_data = analysis_results.get('time', range(len(analysis_results['data'])))
                ax.plot(time_data, analysis_results['data'], linewidth=2)
                ax.set_xlabel('Time (s)')
                ax.set_ylabel('Sensor Value')
                ax.set_title(f'{analysis_results.get("sensor", "Sensor").upper()} Sensor Data')
                ax.grid(True, alpha=0.3)
                plt.tight_layout()
                show_figure(fig)
        
        # If no visualization matched, show message
        if not (mode == 'parameter' and viz == 'results') and not (mode == 'quality' and viz == 'results') and not (mode == 'sensor' and viz == 'results'):
            if not analysis_results and not optimization_results:
                display(HTML("<p>Execute analysis or optimization to see visualization</p>"))
            elif viz == 'optimization' and not optimization_results:
                display(HTML("<p>Run optimization first to see optimization visualization</p>"))
            elif viz == 'pareto' and (not optimization_results or optimization_results.get('type') != 'multi'):
                display(HTML("<p>Run multi-objective optimization first to see Pareto front</p>"))
            else:
                display(HTML(f"<p>Visualization not available for mode: <b>{mode}</b>, view: <b>{viz}</b></p>"))

# Connect events
execute_button.on_click(execute_analysis)
optimize_button.on_click(run_optimization)
viz_mode.observe(lambda x: update_visualization(), names='value')
analysis_mode.observe(lambda x: update_visualization(), names='value')

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to perform process analysis and parameter optimization.

### Key Takeaways

1. **Parameter Analysis**: Distribution, correlation, and interaction analysis
2. **Quality Prediction**: Linear, polynomial, ML, and neural network models
3. **Sensor Analysis**: ISPM and CT sensor data analysis with trend and anomaly detection
4. **Single-Objective Optimization**: Genetic Algorithm, Particle Swarm, Gradient Descent, Bayesian methods
5. **Multi-Objective Optimization**: Pareto front analysis and trade-off visualization
6. **Optimization Visualization**: Progress plots, convergence analysis, Pareto fronts

### Next Steps

Proceed to:
- **12_Virtual_Experiments.ipynb** - Learn to plan and execute virtual experiments
- **13_Anomaly_Detection.ipynb** - Learn anomaly detection techniques

### Related Resources

- Process Analysis Documentation: `../docs/AM_QADF/05-modules/analytics.md`
- API Reference: `../docs/AM_QADF/06-api-reference/analytics-api.md`
- Examples: `../examples/`
